In [10]:
# https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
analyzer = SentimentIntensityAnalyzer()

## Helper functions
### NLTK Application

In [4]:
# the columns are beeing merged togheter here
def merge_columns_in_dataframe(df):
    columns_to_merge = ['page_title', 'sub_title', 'introduction', 'summary_box', 'content', 'accordion']
    new_df = df.loc[:, ["url", "page_title"]].copy()
    new_df['text'] = df.apply(lambda row: ' '.join([str(row[col]) for col in columns_to_merge]), axis=1)
    return new_df

In [12]:
def nltk_text_preprocessing(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

In [13]:
def get_sentiment(text_string):
    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

def add_sentiment_cols(dataframe, text_col="text"):
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [14]:
# def get_analysis_data(json_file):
#     pandas_df = pd.read_json(json_file)
#     filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
#     data = merge_columns_in_dataframe(filtered_df)
#     data["text"] = data["text"].apply(preprocess_text)
#     return add_sentiment_cols(data)

In [ ]:
def get_analysis_data(json_file, preprocessing_method):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = merge_columns_in_dataframe(filtered_df)
    if preprocessing_method == "nltk":
        data["text"] = data["text"].apply(nltk_text_preprocessing)
    elif preprocessing_method == "spacy":
        data["text"] = data["text"].apply(spacy_text_preprocessing)
    else:
        "please provide preprocessing method (nltk/spacy)"
    return add_sentiment_cols(data)

### SpaCy Lemmatization

In [ ]:
def spacy_text_preprocessing(text):
    import spacy
    nlp = spacy.load("de_core_news_sm")

    doc = nlp(text.lower())
    filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

    lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
    processed_text = " ".join(lemmatized_tokens)
    return processed_text

### Huggingface DistillBERT Model Application

In [6]:
import pandas as pd
from transformers import pipeline
def get_sentiment_hf(text_string):
    classifier = pipeline(
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        top_k=None,
        truncation=True,
    )
    result_in_list_list_of_dicts_of_tuples = classifier(text_string)
    mydict = {}
    for result_in_list_of_dicts_of_tuples in result_in_list_list_of_dicts_of_tuples:
        for dicts in result_in_list_of_dicts_of_tuples:
            mydict[dicts["label"]] = dicts["score"]
    return pd.Series(mydict)

def add_sentiment_cols_hf(dataframe, text_col="text"):
    sentiment_cols = dataframe[text_col].apply(get_sentiment_hf)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    return dataframe

/Users/romanroth/opt/anaconda3/envs/WBTH-Project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def get_analysis_data_distilbert(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = merge_columns_in_dataframe(filtered_df)
    return add_sentiment_cols_hf(data)

In [9]:
def save_as_json(pandas_dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    pandas_dataframe.to_json(f"{file_name}_{date_info}.json", orient='records')

In [10]:
# df = get_analysis_data("ratgeber_pages_part2.json")
# save_as_json(df, "ratgeber_pages_with_rating")

## Execution of Code

In [ ]:
# df2 = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_2.json")
# save_as_json(df2, "full_scrape_2_analysis")

In [12]:
df2 = pd.read_json("full_scrape_2_analysis_15-03-24_15:40:08.json")
display(df2)

,url,page_title,text,neg,neu,pos,compound,judgement
0,https://www.mobiliar.ch/versicherungen-und-vor...,Tierversicherung,tierversicher hund katz versich tierversicher ...,0.000,0.994,0.006,0.2406,positive
1,https://www.mobiliar.ch/versicherungen-und-vor...,Cyberversicherung,cyberversicher hilf digital alltag bankgeschaf...,0.028,0.954,0.019,-0.5362,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Wertsachenversicherung,"wertsachenversicher versich , bedeutet wertvol...",0.000,0.981,0.019,0.6553,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Die Hausratversicherung,hausratversicher rundum-versicher beweg sach w...,0.021,0.979,0.000,-0.6662,negative
4,https://www.mobiliar.ch/versicherungen-und-vor...,Mietkautionsversicherung,mietkautionsversicher dank mietkaution spar mi...,0.007,0.976,0.017,0.4329,positive
...,...,...,...,...,...,...,...,...
1456,https://www.mobiliar.ch/versicherungen-und-vor...,Die Fahrrad-Kaskoversicherung vermitteln,fahrrad-kaskoversicher vermitteln umfass versi...,0.028,0.972,0.000,-0.3313,negative
1457,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2022: Insgesamt 19 weitere Brücken un...,projekt 2022 : insgesamt 19 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1458,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2023: Insgesamt 13 weitere Brücken un...,projekt 2023 : insgesamt 13 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1459,https://www.mobiliar.ch/versicherungen-und-vor...,Winter Wanderland,"wint wanderland verschneit tal , weit nebelme ...",0.000,1.000,0.000,0.0000,neutral


In [13]:
df2["judgement"].value_counts()

judgement
neutral     726
positive    465
negative    270
Name: count, dtype: int64

In [7]:
get_analysis_data_distilbert("hans.json")

,url,page_title,text,negative,positive,neutral
1,https://www.mobiliar.ch/versicherungen-und-vor...,Selbstunfall mit dem Auto – was nun?,Selbstunfall mit dem Auto – was nun? Selbst s...,0.785902,0.107085,0.107012
2,https://www.mobiliar.ch/versicherungen-und-vor...,Invalidität oder Todesfall: So sorgen Sie vor,Invalidität oder Todesfall: So sorgen Sie vor ...,0.488104,0.375045,0.136852
3,https://www.mobiliar.ch/versicherungen-und-vor...,Schäden an Ihrer Mietwohnung,Schäden an Ihrer Mietwohnung Für welche Schäde...,0.795895,0.108779,0.095326


In [129]:
get_analysis_data("hans.json")

,url,page_title,text,neg,neu,pos,compound,judgement
1,https://www.mobiliar.ch/versicherungen-und-vor...,Selbstunfall mit dem Auto – was nun?,selbstunfall auto – ? schuld ! sekund aufgepas...,0.025,0.975,0.000,-0.5627,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Invalidität oder Todesfall: So sorgen Sie vor,"invaliditat todesfall : sorg optimal vorsorg ,...",0.000,0.963,0.037,0.7820,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Schäden an Ihrer Mietwohnung,schad mietwohn schad haft mieterin miet ? freu...,0.039,0.961,0.000,-0.9749,negative


## to display the Functions

In [ ]:

import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
analyzer = SentimentIntensityAnalyzer()

In [30]:
text = "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert."

In [50]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(input_text.lower())
output_text = " ".join(tokens)
output_text

'der neue « liebe mobiliar » -spot zeigt , dass schäden zu hause immer genau dann passieren , wenn man nicht damit rechnet .'

In [37]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text.lower())
output_text = " ".join(tokens)
output_text

"damit die freude am neuen occasion-auto nicht von bösen überraschungen ausgebremst wird , lohnt sich beim kauf genaues hinsehen . erfahren sie , wie sie versteckte fahrzeugschäden finden und wer bei verschwiegenen automängel haftesummary_box car check – der digitale helfer für den occasion-kauf beim occasion-kauf gibt es viele dinge zu beachten . besonders , wenn man kein auto-profi ist , kann dies schnell mal überfordern . hier unterstützt der car check der mobiliar : prüfen sie den zustand von gebrauchtwagen einfach und selbstständig . der car check führt sie schritt für schritt durch alle wichtigen punkte und hilft ihnen bei der einschätzung des zustands des occasion-autos . probieren sie es gleich aus ! car check starten die wichtigsten fragen beim occasionkauf was kostet mich ein auto ? gebrauchtwagen vom händler oder privaten verkäufer ? wie kann ich mich bei einem autokauf absichern ? wieso ist eine zustandsprüfung wichtig ? was kann ich auch ohne autokenntnisse prüfen ? wie fü

In [60]:
from nltk.corpus import stopwords
filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
output_text = " ".join(filtered_tokens)
output_text

'neue « liebe mobiliar » -spot zeigt , schäden hause immer genau passieren , rechnet . mobiliar startet neusten « liebe mobiliar » -spot neue jahr . familie spot hingegen startet ersten blick gewöhnlichen sonntagmorgen . : eltern gemütlich hinschlummern , geräuschen geweckt , definitiv haus gehören . grund dafür tochter , talent eiskunstläuferin entdeckt freude auslebt . hinterlässt spuren . immer kommt – helfen rasch unkompliziert .'

In [61]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german")
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
output_text_nltk = " ".join(stemmed_tokens)
output_text_nltk

'neu « lieb mobiliar » -spot zeigt , schad haus imm genau passi , rechnet . mobiliar startet neust « lieb mobiliar » -spot neu jahr . famili spot hingeg startet erst blick gewohn sonntagmorg . : elt gemut hinschlumm , gerausch geweckt , definitiv haus gehor . grund dafur tocht , talent eiskunstlauferin entdeckt freud auslebt . hinterlasst spur . imm kommt – helf rasch unkompliziert .'

In [80]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_nltk)
pd.Series(scores)

neg         0.0000
neu         0.9460
pos         0.0540
compound    0.4215
dtype: float64

In [77]:
import spacy
nlp = spacy.load("de_core_news_sm")

doc = nlp(text.lower())
filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
output_text_spacy = " ".join(lemmatized_tokens)
output_text_spacy

'-- liebe mobiliar»-spot zeigen -- schäden Hause genau passieren -- rechnen --   Mobiliar starten neu -- liebe mobiliar»-spot -- Familie Spot hingegen starten Blick gewöhnlich Sonntagmorg -- -- Eltern gemütlich hinschlummern -- geräuschen wecken -- definitiv Haus gehören -- Grund Tochter -- Talent eiskunstläuferin entdecken Freude ausleben -- Hinterlässt Spur -- -- helfen rasch unkompliziert --'

In [79]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_spacy)
pd.Series(scores)

neg         0.0000
neu         0.9520
pos         0.0480
compound    0.4215
dtype: float64

In [33]:
def calculate_average_scores(sentiment_results):
    scores = {'negative': [], 'neutral': [], 'positive': []}
    for result in sentiment_results:
        for res in result:
            scores[res['label']].append(res['score'])
    average_scores = {label: sum(score_list) / len(score_list) if score_list else 0 for label, score_list in scores.items()}
    return average_scores

In [34]:
from transformers import pipeline
import spacy

classifier = pipeline('sentiment-analysis',
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=None,
    truncation=False,
)

nlp = spacy.load('de_core_news_sm')
doc = nlp(text)
tokens = [token.text for token in doc]
results = []

if len(tokens) > 512:
    for sentence in doc.sents:
        result = classifier(sentence.text)
        results.extend(result)
else:
    result = classifier(text)
    results.extend(result)


average_scores = calculate_average_scores(results)
pd.Series(average_scores)

negative    0.642108
neutral     0.085077
positive    0.272815
dtype: float64

In [31]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis',
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=None,
    truncation=False,
)
result_in_list_list_of_dicts_of_tuples = classifier(text)
mydict = {}
for result_in_list_of_dicts_of_tuples in result_in_list_list_of_dicts_of_tuples:
    for dicts in result_in_list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
pd.Series(mydict)

#ausgegeben werden confidenzscores 

negative    0.642108
positive    0.272815
neutral     0.085077
dtype: float64

## Training the Model
Habe ich nicht umgesetzt.

In [5]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch.nn.functional as F
import torch

model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
classifier = pipeline(
    model=model_name,
    tokenizer = AutoTokenizer.from_pretrained(model_name),
    top_k=None
)

X_train = ["Ich bin Roman und 28 Jahre alt.", "Ich liebe dich!"]
res = classifier(X_train)

print(res)
tokenizer = AutoTokenizer.from_pretrained(model_name)
batch = tokenizer(X_train, padding=True, truncation=True, max_length=800, return_tensors="pt")

with torch.no_grad():
    outputs = model_name(**batch),
    print(outputs)
    predictions = F.softmax(output.logits, dim=1),
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)


# list_list_of_dicts_of_tuples = res
# for list_of_dicts_of_tuples in list_list_of_dicts_of_tuples:
#     for dicts in list_of_dicts_of_tuples:
#         mydict[dicts["label"]] = dicts["score"]
# print(pd.Series(mydict))

[[{'label': 'positive', 'score': 0.49148350954055786}, {'label': 'neutral', 'score': 0.25997069478034973}, {'label': 'negative', 'score': 0.2485458105802536}], [{'label': 'positive', 'score': 0.9740146994590759}, {'label': 'neutral', 'score': 0.015805009752511978}, {'label': 'negative', 'score': 0.01018031220883131}]]


TypeError: 'str' object is not callable